Your goal is to:
- Detect the face in the first frame of the movie
- Using pre-trained Viola-Jones detector
- Track the face throughout the movie using:
   - CAMShift
   - Particle Filter
   - Face detector + Kalman Filter (always run the kf.predict(), and run kf.correct() when you get a new face detection)
- Bonus (20pt): Face Detector + Optical Flow tracker (use the OF tracker whenever the face detector fails).


In [1]:
import os
import sys
import cv2
import matplotlib.pyplot as plt
import numpy as np

face_cascade = cv2.CascadeClassifier('/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml')

Debug_laptop = 1
filename = "02-1.avi"
if Debug_laptop:
    import skvideo.io
    v = skvideo.io.vread(filename)
else:
    v = cv2.VideoCapture(filename)

/Users/trungnguyen/anaconda/lib/python2.7/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)


In [2]:
def detect_one_face(im):
    gray=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.2, 3)
    if len(faces) == 0:
        return (0,0,0,0)
    return faces[0]

In [3]:
def hsv_histogram_for_window(frame, window):
    # set up the ROI for tracking
    c,r,w,h = window
    roi = frame[r:r+h, c:c+w]
    hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
    roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
    cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
    return roi_hist

In [4]:
def resample(weights):
    n = len(weights)
    indices = []
    C = [0.] + [sum(weights[:i+1]) for i in range(n)]
    u0, j = np.random.random(), 0
    for u in [(u0+i)/n for i in range(n)]:
      while u > C[j]:
          j+=1
      indices.append(j-1)
    return indices

In [7]:
output_name = "output_camshift_IPython.txt"
output = open(output_name,"w")
frameCounter = 0
# read first frame

frame = v[0]            # first frame

# detect face in first frame
c,r,w,h = detect_one_face(frame)
faces = (c,r,w,h)
pt = (frameCounter, c + w / 2, r + h / 2)
output.write("%d,%d,%d\n" % pt)  
frameCounter = frameCounter + 1

while (1):
    if frameCounter == len(v): break
    frame = v[frameCounter]  # read frame
    
    # set up the ROI for tracking
    track_window = faces
    roi_hist = hsv_histogram_for_window(frame, (c, r, w, h))  # this is provided for you
    
    # Setup the termination criteria, either 10 iteration or move by atleast 1 pt
    term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
    # apply meanshift to get the new location
    ret, track_window = cv2.CamShift(dst, track_window, term_crit)

    (c,r,w,h) = track_window
    pt = (frameCounter, c + w / 2, r + h / 2)
    frameCounter = frameCounter + 1
    output.write("%d,%d,%d\n" % pt)  
    
        # Draw it on image
    pts = cv2.boxPoints(ret)
    pts = np.int0(pts)
    img2 = cv2.polylines(frame, [pts], True, 255, 2)
    cv2.imshow('img2', img2)
    dst_draw = cv2.polylines(dst, [pts], True, 255, 2)
    # cv2.imshow("BackProject", dst_draw)
    cv2.waitKey(10)

cv2.destroyAllWindows()
output.close()

(0, 91, 68)
